In [ ]:
# === leitura_serial + conversão WFDB com anotações ===
import serial
import time
import numpy as np
import wfdb
from datetime import datetime
from scipy.signal import butter, find_peaks, sosfiltfilt

# === CONFIGURAÇÕES SERIAL E ARQUIVO ===
porta_serial = 'COM7'
baudrate = 115200
tempo_coleta_seg = 15
taxa_amostragem = 977
arquivo_txt = 'ecg_bruto.txt'
arquivo_wfdb = 'meu_ecg_filtrado'

# === CONFIGURAÇÕES DO ADC E SINAL ===
adc_bits = 12 
adc_max = 2**adc_bits - 1  # 4095
v_ref = 3.3  # volts
ganho_adc = 1  # 1 unidade = 5 µV → evita estouro com fmt='16'

# === FILTRO PASSA-FAIXA ===
def filtro_passa_faixa(signal, fs, lowcut=0.5, highcut=45.0, ordem=2):
    nyq = 0.5 * fs
    sos = butter(ordem, [lowcut / nyq, highcut / nyq], btype='band', output='sos')
    return sosfiltfilt(sos, signal)

# === LEITURA SERIAL ===
num_amostras = tempo_coleta_seg * taxa_amostragem
valores = []

print(f"⏳ Preparando conexão com {porta_serial} a {baudrate} baud...")

# Abre a porta sem resetar o ESP32
ser = serial.Serial()
ser.port = porta_serial
ser.baudrate = baudrate
ser.timeout = 1
ser.dtr = False
ser.rts = False
ser.open()
time.sleep(2)  # aguarda estabilização

print(f"📡 Coletando {tempo_coleta_seg} segundos de sinal ({num_amostras} amostras)...")
while len(valores) < num_amostras:
    try:
        linha = ser.readline().decode(errors='ignore').strip()
        valor = int(linha)
        if 0 <= valor <= adc_max:
            valores.append(valor)
        if len(valores) % 1000 == 0:
            print(f"  → {len(valores)} amostras coletadas...")
    except:
        continue

ser.close()

print(f"🔎 Primeiras 10 amostras: {valores[:10]}")
print(f"📊 Estatísticas — min: {np.min(valores)}, max: {np.max(valores)}, média: {np.mean(valores)}")

# === SALVAMENTO COMO TXT ===
print(f"💾 Salvando {len(valores)} amostras em {arquivo_txt}...")
with open(arquivo_txt, 'w') as f:
    for v in valores:
        f.write(f"{v}\n")
